# PROJECT CAPSTONE - CLUSTERING THE WORLD CAPITALS

## 1. BACK GROUND

### 1.1. Introduction


Sister cities or twin towns is a known concept in today’s world where two or more cities or countries form a sort of agreement to promote their ties. These cities or countries need not be in the same geographical areas.  The concept is mainly aimed at increasing the friendship and understanding between the cities or countries that ultimately helps in increasing trade and tourism. Further details of this concept can be seen on this link. 
In this report, we try to find sister cities for world capitals. We select several capital cities across different continents and try to form groups of sister cities using a popular machine learning clustering algorithm and by using the location data of these cities. 

### 1.2. Research Problem

Currently, the concept of sister cities is more driven by political or social ties between cities or countries. The objective of this report is to form clusters of capital cities that are closer to each other based on some physical attributes.

### 1.3. Interests

This piece of research can be of interest for many different groups some of which are as follow;
(a)	Tourists or travelers,
(b)	Tourism companies or authorities,
(c)	Traders/Trading companies & 
(d)	International students

In [4]:
# first let's imposrt all necessary packages required for this analyses

import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim 

import requests
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans
import folium

print('Required libraries imported.')

Required libraries imported.


## 2. DATA

We will need two data sets for this reseach report.  
1. Getet a list of all the world capital cities with country name, city name and longitude/latitude information
2. Using this latitude & longitude information, we will get location or venue data for these capital cities using Foursquare API based on which cluster analyses will be performed

### 2.1. Get the World Capital Cities Data

The data is avaibale on Kaggle (https://www.kaggle.com/nikitagrec/world-capitals-gps). The data has country name, capital name of the country, latitude and longitude of the capital cities, country code and the continent name. I already downloaded the csv file (named: concap.csv) from the given link and saved on my computer's hard drive.

In [113]:
concap_data = pd.read_csv("C://Users//khadmu01//Downloads//world-capitals-gps//concap.csv")
print(concap_data.head())
print(concap_data.shape)

                                CountryName        CapitalName  \
0                                Somaliland           Hargeisa   
1  South Georgia and South Sandwich Islands  King Edward Point   
2       French Southern and Antarctic Lands  Port-aux-Français   
3                                 Palestine          Jerusalem   
4                             Aland Islands          Mariehamn   

   CapitalLatitude  CapitalLongitude CountryCode ContinentName  
0         9.550000         44.050000         NaN        Africa  
1       -54.283333        -36.500000          GS    Antarctica  
2       -49.350000         70.216667          TF    Antarctica  
3        31.766667         35.233333          PS          Asia  
4        60.116667         19.900000          AX        Europe  
(245, 6)


### Plot the world capital cities on the map

In [18]:
world_map = folium.Map(location=[200, -200], zoom_start=1.5)

for lat, lng, cap, cont in zip(concap_data['CapitalLatitude'], concap_data['CapitalLongitude'], concap_data['CapitalName'], concap_data['CountryName']):
    label = '{}, {}'.format(cap, cont)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(world_map)  
    
world_map

The file has data for 245 capital cities. Given that some of these cities will be very small and some may not even have Foursquare data available, so we will be working on a subset of this data. Let's select first 150 capital cities only given in this data

In [116]:
concap_data2 = concap_data[0:150]
concap_data2.shape

(150, 6)

Let's plot these 150 cities on the map

In [147]:
capital_map = folium.Map(location=[200, -200], zoom_start=1.5)

for lat, lng, cap, cont in zip(concap_data2['CapitalLatitude'], concap_data2['CapitalLongitude'], concap_data2['CapitalName'], concap_data2['CountryName']):
    label = '{}, {}'.format(cap, cont)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(capital_map)  
    
capital_map

### 2.2. Get Location Data from Foursquare

Since we have selected the capital cities that we will be working on, we now will move to extract their location data from Foursquare API using latitude & longitude information.

In [31]:
CLIENT_ID = '' # removed because of confidentiality
CLIENT_SECRET = '' # removed because of confidentiality
VERSION = '' # Foursquare API version


In [32]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius


We are using a limit of 100 venuues within the radius of 1000 meter so that we can have reasonable (and if possible closer to each other) number of venues for each city. If we don't apply this condition, then some may cities may have a very large number of venue information while some others may have a very small number. This can impact the clutering results. 

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [49]:
# function that extracts the location data for all the capital cities in the list

def getNearbyVenues(country, capital, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for con, cap, lat, lng in zip(country, capital, latitudes, longitudes):
        print(cap)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            con,
            cap, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Country',
                    'Capital',
                    'Capital_Latitude',
                    'Capital_Longitude',
                    'Venue',
                    'Venue_Latitude',
                    'Venue_Longitude',
                    'Venue_Category']
    
    return(nearby_venues)

Now, let's extract the location data for all 150 capital cities

In [117]:
worldcapital_venues2 = getNearbyVenues(country = concap_data2['CountryName'], capital=concap_data2['CapitalName'],
                                   latitudes=concap_data2['CapitalLatitude'],
                                   longitudes=concap_data2['CapitalLongitude']
                                  )

Hargeisa
King Edward Point
Port-aux-Français
Jerusalem
Mariehamn
Yaren
Marigot
Atafu
El-Aaiún
Kabul
Tirana
Algiers
Pago Pago
Andorra la Vella
Luanda
The Valley
Saint John's
Buenos Aires
Yerevan
Oranjestad
Canberra
Vienna
Baku
Nassau
Manama
Dhaka
Bridgetown
Minsk
Brussels
Belmopan
Porto-Novo
Hamilton
Thimphu
La Paz
Sarajevo
Gaborone
Brasilia
Road Town
Bandar Seri Begawan
Sofia
Ouagadougou
Rangoon
Bujumbura
Phnom Penh
Yaounde
Ottawa
Praia
George Town
Bangui
N'Djamena
Santiago
Beijing
The Settlement
West Island
Bogota
Moroni
Kinshasa
Brazzaville
Avarua
San Jose
Yamoussoukro
Zagreb
Havana
Willemstad
Nicosia
Prague
Copenhagen
Djibouti
Roseau
Santo Domingo
Quito
Cairo
San Salvador
Malabo
Asmara
Tallinn
Addis Ababa
Stanley
Torshavn
Suva
Helsinki
Paris
Papeete
Libreville
Banjul
Tbilisi
Berlin
Accra
Gibraltar
Athens
Nuuk
Saint George's
Hagatna
Guatemala City
Saint Peter Port
Conakry
Bissau
Georgetown
Port-au-Prince
Vatican City
Tegucigalpa
Budapest
Reykjavik
New Delhi
Jakarta
Tehran
Baghdad
Dub

In [118]:
worldcapital_venues2.head()

,Country,Capital,Capital_Latitude,Capital_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Somaliland,Hargeisa,9.550000,44.050000,Hiddo - Dhawr,9.551411,44.047806,Comfort Food Restaurant
1,South Georgia and South Sandwich Islands,King Edward Point,-54.283333,-36.500000,جزيرة سندويشة,-54.282935,-36.495176,Beer Bar
2,South Georgia and South Sandwich Islands,King Edward Point,-54.283333,-36.500000,Bilinmeyen Yer,-54.281560,-36.506960,Racetrack
3,South Georgia and South Sandwich Islands,King Edward Point,-54.283333,-36.500000,Tang Ke Lek Harbour,-54.280980,-36.508610,Harbor / Marina
4,French Southern and Antarctic Lands,Port-aux-Français,-49.350000,70.216667,Book Time,-49.352470,70.218711,Bookstore


In [119]:
worldcapital_venues2.shape

(5913, 8)

Let's check how many venues are available for each city

In [125]:
worldcapital_venues2.groupby(['Capital']).count()


,Country,Capital_Latitude,Capital_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
Capital,,,,,,,
Accra,3,3,3,3,3,3,3
Addis Ababa,3,3,3,3,3,3,3
Algiers,8,8,8,8,8,8,8
Amman,100,100,100,100,100,100,100
Andorra la Vella,47,47,47,47,47,47,47
Antananarivo,11,11,11,11,11,11,11
Asmara,9,9,9,9,9,9,9
Astana,81,81,81,81,81,81,81
Athens,100,100,100,100,100,100,100


As shown in above table, there are many cities with very few venues available. For example, Accra and Addid Ababa both have only 3 venues available while Amman and Athen have 100 venues. This can create a noise in the data and ultimately it can impact the results when we run the k-means algorithm for clustering the capital cities. 
<br>
To address this issue, we select only those cities for our clustering analyses that have more than 60 venues available. This is to ensure a good base for running clustering algorithm.

In [133]:
worldcapital_venues = worldcapital_venues2[worldcapital_venues2.groupby('Capital')['Capital'].transform('count')>60]
worldcapital_venues.head()

,Country,Capital,Capital_Latitude,Capital_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
5,Palestine,Jerusalem,31.766667,35.233333,Mt. Zion Hotel,31.768550,35.226596,Hotel
6,Palestine,Jerusalem,31.766667,35.233333,Mount Zion (הר ציון),31.772212,35.229783,Mountain
7,Palestine,Jerusalem,31.766667,35.233333,City of David,31.774049,35.235800,Historic Site
8,Palestine,Jerusalem,31.766667,35.233333,yes PLANET,31.762070,35.226095,Movie Theater
9,Palestine,Jerusalem,31.766667,35.233333,Jerusalem Cinematheque (סינמטק ירושלים),31.770279,35.226014,Indie Movie Theater


Just to double check that all the selected cities now have more than 60 venues

In [134]:
worldcapital_venues.groupby(['Capital']).count()

,Country,Capital_Latitude,Capital_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
Capital,,,,,,,
Amman,100,100,100,100,100,100,100
Astana,81,81,81,81,81,81,81
Athens,100,100,100,100,100,100,100
Baku,97,97,97,97,97,97,97
Berlin,100,100,100,100,100,100,100
Bishkek,82,82,82,82,82,82,82
Brussels,100,100,100,100,100,100,100
Budapest,100,100,100,100,100,100,100
Cairo,61,61,61,61,61,61,61


In [135]:
#number of unique categories and capital cities
print('There are {} uniques categories.'.format(len(worldcapital_venues['Venue_Category'].unique())))
print('There are {} uniques capitals cities.'.format(len(worldcapital_venues['Capital'].unique())))

There are 398 uniques categories.
There are 50 uniques capitals cities.


Finally, we settle with 50 cities each having between 60 to 100 venues. Total unique category venues are 398.

## 3. CLUSTERING

### 3.1. Prepare data for Clustering

Clustering will be done based on the Venue category data but that is a categorical varibale that cannot beused for clustering as is. We need to transform this categorical variable into numeric or quatitative variable. For this we will use one-hot-encoding techniqu. 

In [136]:
# one hot encoding
worldcapital_onehot = pd.get_dummies(worldcapital_venues[['Venue_Category']], prefix="", prefix_sep="")

# add Capital City column back to dataframe
worldcapital_onehot['Capital'] = worldcapital_venues['Capital']

# move Capital City column to the first column
fixed_columns = [worldcapital_onehot.columns[-1]] + list(worldcapital_onehot.columns[:-1])
worldcapital_onehot = worldcapital_onehot[fixed_columns]

worldcapital_onehot.head()

,Capital,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auvergne Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belarusian Restaurant,Belgian Restaurant,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat Rental,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brasserie,Bratwurst Joint,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Bulgarian Restaurant,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,City Hall,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cretan Restaurant,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Curling Ice,Currywurst Joint,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dizi Place,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fish Taverna,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gilaki Restaurant,Gluten-free Restaurant,Go Kart Track,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Henan Restaurant,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Janguh Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kafenio,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Luggage Store,Magirio,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Night Market,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Ouzeri,Padangnese Restaurant,Paella Restaurant,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian P

After one-hot-encoding, next let's group rows by Capital cities by taking the mean of the frequency of occurrence of each category

In [137]:
worldcapital_grouped = worldcapital_onehot.groupby('Capital').mean().reset_index()
worldcapital_grouped

,Capital,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auvergne Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belarusian Restaurant,Belgian Restaurant,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat Rental,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brasserie,Bratwurst Joint,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Bulgarian Restaurant,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,City Hall,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cretan Restaurant,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Curling Ice,Currywurst Joint,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dizi Place,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fish Taverna,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gilaki Restaurant,Gluten-free Restaurant,Go Kart Track,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Henan Restaurant,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Janguh Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kafenio,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Luggage Store,Magirio,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Night Market,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Ouzeri,Padangnese Restaurant,Paella Restaurant,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian P

In [138]:
worldcapital_grouped.shape

(50, 399)

The data is now ready to run cluster algorithm. However, let's also map city wise most common venues. This will help us later in explaining the cluster results by comparing capital cities in different clusters according to most common venues.

In [139]:
# function to get the most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [140]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Capital']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
worldcapital_venues_sorted = pd.DataFrame(columns=columns)
worldcapital_venues_sorted['Capital'] = worldcapital_grouped['Capital']

for ind in np.arange(worldcapital_grouped.shape[0]):
    worldcapital_venues_sorted.iloc[ind, 1:] = return_most_common_venues(worldcapital_grouped.iloc[ind, :], num_top_venues)

worldcapital_venues_sorted.head()

,Capital,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Amman,Café,Middle Eastern Restaurant,Italian Restaurant,Historic Site,Bookstore,Breakfast Spot,Arts & Crafts Store
1,Astana,Coffee Shop,Café,Restaurant,Italian Restaurant,Electronics Store,Karaoke Bar,Diner
2,Athens,Bar,Café,Coffee Shop,Dessert Shop,Theater,Cocktail Bar,Bookstore
3,Baku,Restaurant,Tea Room,Hotel,Café,Coffee Shop,Turkish Restaurant,Eastern European Restaurant
4,Berlin,Hotel,History Museum,Plaza,Museum,Art Gallery,Art Museum,Concert Hall


### 3.2. Perform k-menas Clustering

In [141]:
# set number of clusters
kclusters = 5

worldcapital_grouped_clustering = worldcapital_grouped.drop('Capital', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(worldcapital_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 2, 3, 2, 4, 1, 3, 3, 0, 3, 2, 3, 3, 4, 3, 0, 2, 4, 3, 2, 1, 3,
       1, 3, 1, 4, 1, 2, 3, 2, 3, 4, 4, 4, 3, 3, 3, 2, 1, 2, 3, 3, 4, 0,
       2, 4, 1, 4, 2, 2])

Now we will add cluster labels into our data frame where we have arranged capital cities with their most common venue

In [142]:
# add clustering labels
worldcapital_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
worldcapital_venues_sorted.head()

,Cluster Labels,Capital,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,0,Amman,Café,Middle Eastern Restaurant,Italian Restaurant,Historic Site,Bookstore,Breakfast Spot,Arts & Crafts Store
1,2,Astana,Coffee Shop,Café,Restaurant,Italian Restaurant,Electronics Store,Karaoke Bar,Diner
2,3,Athens,Bar,Café,Coffee Shop,Dessert Shop,Theater,Cocktail Bar,Bookstore
3,2,Baku,Restaurant,Tea Room,Hotel,Café,Coffee Shop,Turkish Restaurant,Eastern European Restaurant
4,4,Berlin,Hotel,History Museum,Plaza,Museum,Art Gallery,Art Museum,Concert Hall


Nest step is to map these clusters using Folium. But for this we need latitude & longitude information that is not availabe on above data set. However, that is available in the original data set we downloaded from Kaggle. So the above data frame with cluster lable and the most common venue information is merged with the original world capital cities data with latitude & longitude.

In [143]:
# merge worldcapital_venues_sorted data with the original data having latitude & longitude of the capital cities
worldcapital_merged = concap_data2.join(worldcapital_venues_sorted.set_index('Capital'), on='CapitalName')

worldcapital_merged.head() 

,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,CountryCode,ContinentName,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Somaliland,Hargeisa,9.550000,44.050000,NaN,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,South Georgia and South Sandwich Islands,King Edward Point,-54.283333,-36.500000,GS,Antarctica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,French Southern and Antarctic Lands,Port-aux-Français,-49.350000,70.216667,TF,Antarctica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Palestine,Jerusalem,31.766667,35.233333,PS,Asia,4.0,Historic Site,Hotel,Restaurant,Mediterranean Restaurant,Park,Italian Restaurant,Burger Joint
4,Aland Islands,Mariehamn,60.116667,19.900000,AX,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Since, we did cluster analyses based on 50 cities only that had more that 60 venues (each) so we see may rown with missing values in the above table. So, let's drop all rows with missing values.

In [144]:
# Since, we finally selected 50 cities only that had more that 60 venues (each) so we ave some missing rows as s
worldcapital_merged = worldcapital_merged.dropna(axis=0)
worldcapital_merged['Cluster Labels'] = worldcapital_merged['Cluster Labels'].astype(int)
worldcapital_merged

,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,CountryCode,ContinentName,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
3,Palestine,Jerusalem,31.766667,35.233333,PS,Asia,4,Historic Site,Hotel,Restaurant,Mediterranean Restaurant,Park,Italian Restaurant,Burger Joint
10,Albania,Tirana,41.316667,19.816667,AL,Europe,2,Cocktail Bar,Café,Italian Restaurant,Bar,Hotel,Lounge,Restaurant
19,Aruba,Oranjestad,12.516667,-70.033333,AW,North America,3,Caribbean Restaurant,Bar,Shopping Mall,Hotel,Breakfast Spot,Coffee Shop,Restaurant
21,Austria,Vienna,48.200000,16.366667,AT,Europe,4,Café,Hotel,Plaza,Museum,Asian Restaurant,Bar,Concert Hall
22,Azerbaijan,Baku,40.383333,49.866667,AZ,Europe,2,Restaurant,Tea Room,Hotel,Café,Coffee Shop,Turkish Restaurant,Eastern European Restaurant
23,Bahamas,Nassau,25.083333,-77.350000,BS,North America,3,Boat or Ferry,Seafood Restaurant,Caribbean Restaurant,Bar,Hotel,Beach,Fast Food Restaurant
24,Bahrain,Manama,26.233333,50.566667,BH,Asia,2,Café,Hotel,Breakfast Spot,Italian Restaurant,Coffee Shop,Restaurant,Lounge
27,Belarus,Minsk,53.900000,27.566667,BY,Europe,3,Cocktail Bar,Bar,Restaurant,Café,Park,Coffee Shop,Boutique
28,Belgium,Brussels,50.833333,4.333333,BE,Europe,3,Sandwich Place,Hotel,Bar,Greek Restaurant,Brasserie,Coffee Shop,Vegetarian / Vegan Restaurant
34,Bosnia and Herzegovina,Sarajevo,43.866667,18.416667,BA,Europe,2,Café,Restaurant,Hotel,Italian Restaurant,Hostel,Cocktail Bar,Theater


Let see how many capital cities are there in each cluster

In [150]:
worldcapital_merged.groupby(['Cluster Labels'])['CapitalName'].count()

Cluster Labels
0     4
1     7
2    12
3    17
4    11
Name: CapitalName, dtype: int64

The data is ready to map clusters on world map

In [146]:
# create map
map_clusters = folium.Map(location=[200, -200], zoom_start=2)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(worldcapital_merged['CapitalLatitude'], worldcapital_merged['CapitalLongitude'], worldcapital_merged['CapitalName'], worldcapital_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=1,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 4. RESULTS

### Cluster 1

In [151]:
worldcapital_merged.loc[worldcapital_merged['Cluster Labels'] == 0, worldcapital_merged.columns[[1] + list(range(5, worldcapital_merged.shape[1]))]]

,CapitalName,ContinentName,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
71,Cairo,Africa,0,Café,Plaza,Coffee Shop,Egyptian Restaurant,Theater,Pastry Shop,Hotel Bar
105,Tehran,Asia,0,Café,Persian Restaurant,Coffee Shop,Sandwich Place,Theater,Art Gallery,Pastry Shop
114,Amman,Asia,0,Café,Middle Eastern Restaurant,Italian Restaurant,Historic Site,Bookstore,Breakfast Spot,Arts & Crafts Store
121,Kuwait City,Asia,0,Café,Coffee Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Hookah Bar,Pizza Place


### Cluster 2

In [152]:
worldcapital_merged.loc[worldcapital_merged['Cluster Labels'] == 1, worldcapital_merged.columns[[1] + list(range(5, worldcapital_merged.shape[1]))]]

,CapitalName,ContinentName,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
43,Phnom Penh,Asia,1,Coffee Shop,Chinese Restaurant,Asian Restaurant,Hotel,Japanese Restaurant,Café,Thai Restaurant
45,Ottawa,Central America,1,Coffee Shop,Café,Restaurant,Tapas Restaurant,Hotel,Food Truck,Italian Restaurant
64,Nicosia,Europe,1,Coffee Shop,Greek Restaurant,Bar,Café,Wine Bar,Italian Restaurant,Restaurant
116,Nairobi,Africa,1,Coffee Shop,Café,African Restaurant,Hotel,Bar,Fast Food Restaurant,Ice Cream Shop
119,Seoul,Asia,1,Coffee Shop,Café,Korean Restaurant,Italian Restaurant,Park,Scenic Lookout,French Restaurant
122,Bishkek,Asia,1,Coffee Shop,Café,Asian Restaurant,Hotel,Turkish Restaurant,Japanese Restaurant,Bar
146,Ulaanbaatar,Asia,1,Coffee Shop,Restaurant,Café,Bakery,Italian Restaurant,Pub,Lounge


### Cluster 3

In [153]:
worldcapital_merged.loc[worldcapital_merged['Cluster Labels'] == 2, worldcapital_merged.columns[[1] + list(range(5, worldcapital_merged.shape[1]))]]

,CapitalName,ContinentName,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
10,Tirana,Europe,2,Cocktail Bar,Café,Italian Restaurant,Bar,Hotel,Lounge,Restaurant
22,Baku,Europe,2,Restaurant,Tea Room,Hotel,Café,Coffee Shop,Turkish Restaurant,Eastern European Restaurant
24,Manama,Asia,2,Café,Hotel,Breakfast Spot,Italian Restaurant,Coffee Shop,Restaurant,Lounge
34,Sarajevo,Europe,2,Café,Restaurant,Hotel,Italian Restaurant,Hostel,Cocktail Bar,Theater
61,Zagreb,Europe,2,Café,Restaurant,Bar,Bakery,Hotel,Gym / Fitness Center,BBQ Joint
93,Guatemala City,Central America,2,Café,Pizza Place,Fast Food Restaurant,Restaurant,Coffee Shop,Steakhouse,Burger Joint
115,Astana,Asia,2,Coffee Shop,Café,Restaurant,Italian Restaurant,Electronics Store,Karaoke Bar,Diner
120,Pristina,Europe,2,Restaurant,Bar,Hotel,Dessert Shop,Mediterranean Restaurant,Fast Food Restaurant,Café
123,Vientiane,Asia,2,Hotel,Café,Asian Restaurant,Bar,Coffee Shop,Pizza Place,French Restaurant
132,Skopje,Europe,2,Café,Hotel,Bar,Restaurant,Historic Site,Italian Restaurant,Bookstore


### Cluster 4

In [154]:
worldcapital_merged.loc[worldcapital_merged['Cluster Labels'] == 3, worldcapital_merged.columns[[1] + list(range(5, worldcapital_merged.shape[1]))]]

,CapitalName,ContinentName,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
19,Oranjestad,North America,3,Caribbean Restaurant,Bar,Shopping Mall,Hotel,Breakfast Spot,Coffee Shop,Restaurant
23,Nassau,North America,3,Boat or Ferry,Seafood Restaurant,Caribbean Restaurant,Bar,Hotel,Beach,Fast Food Restaurant
27,Minsk,Europe,3,Cocktail Bar,Bar,Restaurant,Café,Park,Coffee Shop,Boutique
28,Brussels,Europe,3,Sandwich Place,Hotel,Bar,Greek Restaurant,Brasserie,Coffee Shop,Vegetarian / Vegan Restaurant
39,Sofia,Europe,3,Coffee Shop,Restaurant,Vegetarian / Vegan Restaurant,Bakery,Dessert Shop,Park,Café
50,Santiago,South America,3,Bar,Pizza Place,Chinese Restaurant,Martial Arts Dojo,Hostel,Café,Asian Restaurant
59,San Jose,Central America,3,Bar,Sandwich Place,Hotel,Coffee Shop,Ice Cream Shop,Café,Restaurant
65,Prague,Europe,3,Café,Bakery,Vietnamese Restaurant,Bar,Gym / Fitness Center,Hotel,Pub
66,Copenhagen,Europe,3,Italian Restaurant,Café,Scandinavian Restaurant,Bakery,Coffee Shop,Gym / Fitness Center,Ice Cream Shop
69,Santo Domingo,North America,3,Hotel,Pharmacy,Ice Cream Shop,BBQ Joint,Restaurant,Pizza Place,Bar


### Cluster 5

In [155]:
worldcapital_merged.loc[worldcapital_merged['Cluster Labels'] == 4, worldcapital_merged.columns[[1] + list(range(5, worldcapital_merged.shape[1]))]]

,CapitalName,ContinentName,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
3,Jerusalem,Asia,4,Historic Site,Hotel,Restaurant,Mediterranean Restaurant,Park,Italian Restaurant,Burger Joint
21,Vienna,Europe,4,Café,Hotel,Plaza,Museum,Asian Restaurant,Bar,Concert Hall
81,Paris,Europe,4,Hotel,French Restaurant,Plaza,Japanese Restaurant,Historic Site,Theater,Italian Restaurant
85,Tbilisi,Europe,4,Hotel,Caucasian Restaurant,Restaurant,Bed & Breakfast,Bus Station,Supermarket,Metro Station
86,Berlin,Europe,4,Hotel,History Museum,Plaza,Museum,Art Gallery,Art Museum,Concert Hall
109,Jerusalem,Asia,4,Historic Site,Hotel,Restaurant,Mediterranean Restaurant,Park,Italian Restaurant,Burger Joint
110,Rome,Europe,4,Italian Restaurant,Plaza,Ice Cream Shop,Monument / Landmark,Sandwich Place,Boutique,Hotel
112,Tokyo,Asia,4,Historic Site,Soba Restaurant,Convenience Store,Coffee Shop,Hotel,Ramen Restaurant,Japanese Restaurant
113,Saint Helier,Europe,4,Hotel,Coffee Shop,Pub,Restaurant,Department Store,Fish & Chips Shop,Harbor / Marina
124,Riga,Europe,4,Restaurant,Eastern European Restaurant,Hotel,Bar,Plaza,Park,Café
